In [ ]:
# 기존에 subwords 데이터셋으로 만든 모델에 Flatten() 과 평균 풀링이 있는 부분을 LSTM으로 대체했다.(순환 레이어, RNN)
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Single Layer LSTM

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow_datasets as tfds
import tensorflow as tf
print(tf.__version__)

2.4.0


In [2]:
import tensorflow_datasets as tfds
import tensorflow as tf
print(tf.__version__)

2.4.0


In [3]:
# Get the data
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

In [4]:
tokenizer = info.features['text'].encoder # 단어 -> 토큰화 훈련을 완료한 토커나이저를 가져온다. 이래서 굳이 단어 다 긁어와서 토커나이저를 훈련시킬 필요가 없다고 한거구나.

In [5]:
# 데이터 생성
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE) # 데이터 섞기. 데이터에서 BUFFER_SIZE 개를 뽑아 섞고 반환.
train_dataset = train_dataset.padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(train_dataset)) # 패딩(길이 맞춰주기 위함)
test_dataset = test_dataset.padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(test_dataset)) # 

In [6]:
# 모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, 64), # 임베딩 레이어
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)), # RNN 사용(LSTM)
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [7]:
model.summary() # bidirectional의 출력 형태가 128인 것이 보인다. 왜냐하면 양방향이기 때문에 64*2 = 128이 된 것이다. 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          523840    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               66048     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 598,209
Trainable params: 598,209
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
NUM_EPOCHS = 10
history = model.fit(train_dataset, epochs=NUM_EPOCHS, validation_data=test_dataset)

Epoch 1/10
391/391 [==============================] - 367s 933ms/step - loss: 0.6115 - accuracy: 0.6428 - val_loss: 0.5330 - val_accuracy: 0.7325
Epoch 2/10
131/391 [=========>....................] - ETA: 3:30 - loss: 0.3963 - accuracy: 0.8275

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

In [ ]:
plot_graphs(history, 'accuracy')

In [ ]:
plot_graphs(history, 'loss')